## Step 1: Setup and Package Imports

In [ ]:
import duckdb  
import pandas as pd

## Step 2: Load and Preview Dataset

In [ ]:
##Step 1 : Load Dataset + Preview 
import duckdb
import pandas as pd

# Load and preview the dataset
query = """
SELECT * 
FROM '/kaggle/input/ecommerce-dataset/events.csv'
LIMIT 5
"""
duckdb.query(query).df()

## Step 3: Dataset Size

In [ ]:
import duckdb

# Correct way to run SQL query in Python
duckdb.query("""
    SELECT COUNT(*) AS total_rows
    FROM '/kaggle/input/ecommerce-dataset/events.csv'
""").df()

### Unique Visitors and Items

In [ ]:
duckdb.query("""
    SELECT 
        COUNT(DISTINCT visitorid) AS unique_visitors,
        COUNT(DISTINCT itemid) AS unique_items
    FROM '/kaggle/input/ecommerce-dataset/events.csv'
""").df()

## Frequency of Event Types

In [ ]:
duckdb.query("""
    SELECT event, COUNT(*) AS event_count
    FROM '/kaggle/input/ecommerce-dataset/events.csv'
    GROUP BY event
    ORDER BY event_count DESC
""").df()

## Step 4: Visualize Event Type Distribution

In [ ]:
import matplotlib.pyplot as plt

# Create the event distribution table
event_df = duckdb.query("""
    SELECT event, COUNT(*) AS event_count
    FROM '/kaggle/input/ecommerce-dataset/events.csv'
    GROUP BY event
    ORDER BY event_count DESC
""").df()

## Step 5: Timestamp to Date & Hour
Convert epoch timestamps into readable dates and hours.

In [ ]:
# Extract timestamp → readable format
duckdb.query("""
    SELECT 
        timestamp,
        TO_TIMESTAMP(timestamp / 1000) AS event_time,
        STRFTIME(TO_TIMESTAMP(timestamp / 1000), '%Y-%m-%d') AS event_date,
        STRFTIME(TO_TIMESTAMP(timestamp / 1000), '%H') AS hour,
        event
    FROM '/kaggle/input/ecommerce-dataset/events.csv'
    LIMIT 100000
""").df().head()

## Step 6: Daily Event Trends
Visualize how user engagement changes across days.

In [ ]:
daily_df = duckdb.query("""
    SELECT 
        STRFTIME(TO_TIMESTAMP(timestamp / 1000), '%Y-%m-%d') AS event_date,
        COUNT(*) AS total_events
    FROM '/kaggle/input/ecommerce-dataset/events.csv'
    GROUP BY event_date
    ORDER BY event_date
""").df()

# Plot
plt.figure(figsize=(10, 4))
plt.plot(daily_df['event_date'], daily_df['total_events'], color='purple')
plt.xticks(rotation=45)
plt.title('Total Events per Day')
plt.xlabel('Date')
plt.ylabel('Number of Events')
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()

## Step 7: Hourly Engagement Pattern
Peak browsing times throughout the day.

In [ ]:
# Query hourly activity pattern
hourly_df = duckdb.query("""
    SELECT 
        STRFTIME(TO_TIMESTAMP(timestamp / 1000), '%H') AS hour,
        COUNT(*) AS total_events
    FROM '/kaggle/input/ecommerce-dataset/events.csv'
    GROUP BY hour
    ORDER BY hour
""").df()

# Convert hour to int for sorting
hourly_df['hour'] = hourly_df['hour'].astype(int)

# Plot
plt.figure(figsize=(8, 4))
plt.plot(hourly_df['hour'], hourly_df['total_events'], marker='o', color='darkorange', linewidth=2)
plt.title('🕐 Hourly User Activity Trend', fontsize=14)
plt.xlabel('Hour of Day (24h)')
plt.ylabel('Number of Events')
plt.xticks(range(0, 24))
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()

## Step 8: Funnel Metrics per Product

We calculate conversion metrics for each product:
- Views → Cart
- Cart → Purchase
- Views → Purchase

In [ ]:
# Aggregate funnel metrics per itemid
funnel_df = duckdb.query("""
    SELECT 
        itemid,
        SUM(CASE WHEN event = 'view' THEN 1 ELSE 0 END) AS views,
        SUM(CASE WHEN event = 'addtocart' THEN 1 ELSE 0 END) AS add_to_cart,
        SUM(CASE WHEN event = 'transaction' THEN 1 ELSE 0 END) AS purchases
    FROM '/kaggle/input/ecommerce-dataset/events.csv'
    GROUP BY itemid
""").df()

# Calculate funnel ratios
funnel_df['view_to_cart'] = funnel_df['add_to_cart'] / funnel_df['views']
funnel_df['cart_to_buy'] = funnel_df['purchases'] / funnel_df['add_to_cart']
funnel_df['view_to_buy'] = funnel_df['purchases'] / funnel_df['views']

# Replace infinite or NaN values
funnel_df.replace([float('inf'), -float('inf')], pd.NA, inplace=True)
funnel_df.dropna(subset=['view_to_cart', 'cart_to_buy', 'view_to_buy'], inplace=True)

# Preview top results
funnel_df.head()

## Step 9: Filter High-View SKUs

In [ ]:
query = """
SELECT
    itemid,
    SUM(CASE WHEN event = 'view' THEN 1 ELSE 0 END) AS views,
    SUM(CASE WHEN event = 'addtocart' THEN 1 ELSE 0 END) AS add_to_cart,
    SUM(CASE WHEN event = 'transaction' THEN 1 ELSE 0 END) AS purchases
FROM '/kaggle/input/ecommerce-dataset/events.csv'
GROUP BY itemid
HAVING views > 100
ORDER BY views DESC
"""

funnel_df = duckdb.query(query).df()
funnel_df.head()

### Step 10: Calculate Funnel Conversion Rates for High-View SKUs

In [ ]:
funnel_df['view_to_cart'] = funnel_df['add_to_cart'] / funnel_df['views']
funnel_df['cart_to_buy'] = funnel_df['purchases'] / funnel_df['add_to_cart']
funnel_df['view_to_buy'] = funnel_df['purchases'] / funnel_df['views']

# Replace inf or NaNs from division by 0
funnel_df.replace([float('inf'), -float('inf')], 0, inplace=True)
funnel_df.fillna(0, inplace=True)

funnel_df.head(10)

### Step 11: Simulate Price and Cost for Each SKU

In [ ]:
import numpy as np

# Simulate margin info based on funnel_df itemids
margin_df = funnel_df[['itemid']].copy()
np.random.seed(42)

# Simulated price and cost
margin_df['price'] = np.random.uniform(20, 100, size=len(margin_df))
margin_df['cost'] = margin_df['price'] * np.random.uniform(0.5, 0.85, size=len(margin_df))

margin_df['price'] = margin_df['price'].round(2)
margin_df['cost'] = margin_df['cost'].round(2)

margin_df.head()

### Step 12: Merge Funnel & Margin Data + Flag Profit Erosion

In [ ]:
import pandas as pd

# Now calculate margin per unit
margin_df['margin_per_unit'] = margin_df['price'] - margin_df['cost']

# Merge with funnel data
merged_df = pd.merge(funnel_df, margin_df, on='itemid', how='left')

# Calculate gross margin
merged_df['gross_margin'] = merged_df['margin_per_unit'] * merged_df['purchases']

# Flag erosion
merged_df['erosion_flag'] = ((merged_df['views'] > 1000) &
                             (merged_df['view_to_buy'] < 0.01) &
                             (merged_df['margin_per_unit'] > 10)).astype(int)

merged_df.head(10)

### Step 13: Export Dataset

In [ ]:
merged_df.to_csv('profit_erosion_sku_analysis.csv', index=False)

### Step 14: Identify Top Grossing SKUs

In [ ]:
duckdb.query("""
    SELECT itemid, purchases, gross_margin, margin_per_unit
    FROM df
    WHERE purchases > 10
    ORDER BY gross_margin DESC
    LIMIT 10
""").df()

### Step 15: Identify High-Traffic SKUs with Zero Purchases

In [ ]:
duckdb.query("""
    SELECT itemid, views, purchases, view_to_buy, erosion_flag
    FROM df
    WHERE purchases = 0 AND views > 1000
    ORDER BY views DESC
""").df()

### Step 16: SKUs with Maximum Profit Erosion

In [ ]:
duckdb.query("""
    SELECT itemid, views, purchases, view_to_buy, margin_per_unit, gross_margin
    FROM df
    WHERE erosion_flag = 1
    ORDER BY margin_per_unit DESC
""").df()

### Step 17: Erosion Score — Prioritize Most Critical SKUs

In [ ]:
# Normalize helper
from sklearn.preprocessing import MinMaxScaler

# Copy to avoid modifying source
score_df = merged_df.copy()


# Fill NA values just in case
score_df.fillna(0, inplace=True)

# Normalize the key metrics to 0–1
scaler = MinMaxScaler()

score_df['norm_views'] = scaler.fit_transform(score_df[['views']])
score_df['norm_view_to_buy'] = 1 - scaler.fit_transform(score_df[['view_to_buy']])  # invert
score_df['norm_margin'] = scaler.fit_transform(score_df[['margin_per_unit']])
score_df['norm_age'] = scaler.fit_transform(score_df[['views']])  # assume views = time exposed
score_df['cart_no_purchase'] = ((score_df['add_to_cart'] > 0) & (score_df['purchases'] == 0)).astype(int)

# Weighted Score
score_df['erosion_score'] = (
    score_df['norm_views'] * 0.35 +
    score_df['norm_view_to_buy'] * 0.25 +
    score_df['norm_margin'] * 0.25 +
    score_df['cart_no_purchase'] * 0.15
) * 100

# Cleaned output
scored_products = score_df[['itemid', 'views', 'purchases', 'margin_per_unit', 'view_to_buy', 'erosion_score']]
scored_products = scored_products.sort_values(by='erosion_score', ascending=False)

scored_products.head(10)

### Step 18: Estimate Potential Margin Recovery

In [ ]:
# Import margin per unit from the correct dataset
merged_margin_df = merged_df[['itemid', 'margin_per_unit']]
top_erosion_df = pd.merge(top_erosion_df, merged_margin_df, on='itemid', how='left')

# Now calculate margin recovered
top_erosion_df['potential_margin_recovered'] = top_erosion_df['potential_extra_purchases'] * top_erosion_df['margin_per_unit']

# Final view
loss_impact_df = top_erosion_df[['itemid', 'views', 'purchases', 'margin_per_unit',
                                 'potential_extra_purchases', 'potential_margin_recovered']].sort_values(
    by='potential_margin_recovered', ascending=False)

loss_impact_df.head(10)

### Step 19: Erosion Tagging for Qualitative Diagnosis

In [ ]:
# Add tags to score_df (your erosion dataset)

def generate_tag(row):
    if row['view_to_buy'] < 0.001:
        return 'Zero Conversion'
    elif row['view_to_buy'] < 0.005:
        return 'Low Conversion'
    elif row['view_to_buy'] > 0.05 and row['margin_per_unit'] > 20:
        return 'High Margin, Moderate Conversion'
    elif row['margin_per_unit'] > 25 and row['purchases'] == 0:
        return 'High Margin No Sales'
    elif row['views'] > 3000 and row['add_to_cart'] == 0:
        return 'Viewed but Ignored'
    else:
        return 'Unclear'

score_df['erosion_tag'] = score_df.apply(generate_tag, axis=1)

### Step 20: Recommend Actionable Strategies Based on Tags

In [ ]:
def suggest_strategy(tag):
    if tag == 'Zero Conversion':
        return 'Investigate PDP issues / Reposition listing'
    elif tag == 'Low Conversion':
        return 'Price test or update visuals'
    elif tag == 'High Margin No Sales':
        return 'Run micro discount pilot or bundle'
    elif tag == 'Viewed but Ignored':
        return 'Audit product placement / Search result position'
    else:
        return 'Needs manual deep dive'

score_df['strategy'] = score_df['erosion_tag'].apply(suggest_strategy)

### Step 21: Final Strategic Recommendation Sheet

In [ ]:
final_recommendation_df = score_df.sort_values(by='erosion_score', ascending=False)[
    ['itemid', 'views', 'purchases', 'margin_per_unit', 'view_to_buy', 'erosion_score', 'erosion_tag', 'strategy']
].head(15)

final_recommendation_df.head(15)

### Step 22: Establish Conversion Benchmark  

In [ ]:
benchmark_rate = merged_df['view_to_buy'].quantile(0.90)
print(f"Benchmark View-to-Buy Rate: {benchmark_rate:.4f}")

### Step 23: Estimate Potential Extra Purchases  

In [ ]:
merged_df['potential_extra_purchases'] = (
    (merged_df['views'] * benchmark_rate) - merged_df['purchases']
).clip(lower=0)

### Step 24: Calculate Potential Margin Recovered  

In [ ]:
# Step 3: Calculate potential margin recovered
merged_df['potential_margin_recovered'] = (
    merged_df['potential_extra_purchases'] * merged_df['margin_per_unit']
)

### Step 25: Identify Top Erosion SKUs with High Recovery Potential  

In [ ]:
# Step 4: Filter for top erosion cases with potential upside
recovery_df = merged_df[merged_df['erosion_flag'] == 1].copy()

top_recovery = recovery_df[[
    'itemid', 'views', 'purchases', 'margin_per_unit',
    'view_to_buy', 'potential_extra_purchases', 'potential_margin_recovered'
]].sort_values(by='potential_margin_recovered', ascending=False).head(15)

top_recovery

### Step 26: Smart Erosion Score – Advanced Prioritization  

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Copy the merged dataframe
score_df = merged_df.copy()

# Fill any missing values
score_df.fillna(0, inplace=True)

# Flag: Cart Added but No Purchase
score_df['cart_no_purchase'] = ((score_df['add_to_cart'] > 0) & (score_df['purchases'] == 0)).astype(int)

# Initialize scaler
scaler = MinMaxScaler()

# Create normalized columns
score_df['norm_views'] = scaler.fit_transform(score_df[['views']])
score_df['norm_view_to_buy'] = 1 - scaler.fit_transform(score_df[['view_to_buy']])  # invert
score_df['norm_unit_margin'] = scaler.fit_transform(score_df[['margin_per_unit']])
score_df['norm_gross_margin'] = 1 - scaler.fit_transform(score_df[['gross_margin']])  # invert

# Smart erosion score (real-world weights)
score_df['smart_erosion_score'] = (
    score_df['norm_views'] * 0.30 +
    score_df['norm_view_to_buy'] * 0.30 +
    score_df['norm_unit_margin'] * 0.20 +
    score_df['norm_gross_margin'] * 0.10 +
    score_df['cart_no_purchase'] * 0.10
) * 100

# Top at-risk products
top_smart_erosion = score_df[[
    'itemid', 'views', 'purchases', 'margin_per_unit', 'gross_margin', 
    'view_to_buy', 'smart_erosion_score'
]].sort_values(by='smart_erosion_score', ascending=False).head(15)

top_smart_erosion.reset_index(drop=True, inplace=True)
top_smart_erosion

### Step 27: Benchmark Conversion Rate  

In [ ]:
# 90th percentile view-to-buy conversion rate across all SKUs
benchmark_rate = merged_df['view_to_buy'].quantile(0.90)
print(f"Benchmark View-to-Buy Rate: {benchmark_rate:.4f}")

### Step 28: Estimate Potential Extra Purchases  

In [ ]:
# Calculate the expected purchases at benchmark conversion
merged_df['expected_purchases'] = merged_df['views'] * benchmark_rate

# Calculate potential extra purchases
merged_df['potential_extra_purchases'] = merged_df['expected_purchases'] - merged_df['purchases']
merged_df['potential_extra_purchases'] = merged_df['potential_extra_purchases'].apply(lambda x: max(x, 0))  # avoid negatives

### Step 29: Identify High-Upside SKUs  

In [ ]:
uplift_df = merged_df[
    ['itemid', 'views', 'purchases', 'margin_per_unit', 'view_to_buy', 'potential_extra_purchases', 'potential_margin_recovered']
].sort_values(by='potential_margin_recovered', ascending=False)

uplift_df.head(10)

### Step 30: Uplift Simulation — Strategic Recovery Scenarios

In [ ]:
# Define benchmark uplift targets
p90_rate = merged_df['view_to_buy'].quantile(0.90)
top_1p_rate = merged_df['view_to_buy'].quantile(0.99)

# Assume current rate uplift by +50% for Scenario B
def uplift_simulator(df):
    df = df.copy()
    
    df['uplift_A'] = p90_rate
    df['uplift_B'] = df['view_to_buy'] * 1.5
    df['uplift_C'] = top_1p_rate
    
    for col in ['uplift_A', 'uplift_B', 'uplift_C']:
        df[f'expected_extra_purchases_{col[-1]}'] = (df[col] - df['view_to_buy']) * df['views']
        df[f'recovered_margin_{col[-1]}'] = df[f'expected_extra_purchases_{col[-1]}'] * df['margin_per_unit']
    
    return df[[
        'itemid', 'views', 'purchases', 'margin_per_unit', 'view_to_buy',
        'expected_extra_purchases_A', 'recovered_margin_A',
        'expected_extra_purchases_B', 'recovered_margin_B',
        'expected_extra_purchases_C', 'recovered_margin_C'
    ]]

# Run simulator on top erosion SKUs
uplift_df = uplift_simulator(top_erosion_df)
uplift_df.head(10)

### Step 31: Priority Action Tags Based on Recovery Potential

In [ ]:
final_df = uplift_df.copy()

def tag_priority_action(row):
    if row['recovered_margin_A'] >= 3000:
        return '🔴 Critical | Reposition SKU + Fix PDP immediately'
    elif row['recovered_margin_A'] >= 1000:
        return '🟠 High | A/B test offer / urgency CTA'
    elif row['recovered_margin_A'] >= 500:
        return '🟡 Medium | Bundle or reposition in category'
    else:
        return '⚪ Low | Deprioritize or test seasonally'

final_df['action_priority'] = final_df.apply(tag_priority_action, axis=1)

# Preview result
final_df[['itemid', 'recovered_margin_A', 'action_priority']].head(10)

### Step 30: Actionable Promo Strategy Recommendation

In [ ]:
def recommend_action(row):
    # Kill if zero purchases and low recovery potential
    if row['purchases_x'] == 0 and row['max_recovery'] < 300:
        return '❌ Kill SKU'
    
    # Recommend promo cut if recovery is good but performance is weak
    elif row['max_recovery'] >= 300 and row['purchases_x'] < 5:
        return '🔁 Cut Promo (' + row['best_scenario'].split('_')[-1] + ')'
    
    # Keep if purchases decent or promo benefit is marginal
    else:
        return '✅ Keep'

In [ ]:
merged_df['action'] = merged_df.apply(recommend_action, axis=1)

### Step 31: Assign Final Promo-Based Action to Each SKU

In [ ]:
final_actions = merged_df[[
    'itemid', 'views_x', 'purchases_x', 'margin_per_unit_x',
    'max_recovery', 'best_scenario', 'action'
]]

final_actions = final_actions.sort_values(by='max_recovery', ascending=False)
final_actions.head(10)

### Step 32: Final SKU-Level Action Plan

In [ ]:
def confidence_level(row):
    if row['max_recovery'] > 3000 and row['margin_per_unit_x'] > 25:
        return '🔵 High'
    elif row['max_recovery'] > 1000:
        return '🟡 Medium'
    else:
        return '🔴 Low'

merged_df['confidence'] = merged_df.apply(confidence_level, axis=1)

### Step 33: SKU Removal Risk Assessment

In [ ]:
def risk_score(row):
    if row['purchases_x'] > 10 and row['view_to_buy_x'] > 0.03:
        return '⚠️ High Risk to Remove'
    elif row['purchases_x'] <= 2:
        return '✅ Low Risk to Remove'
    else:
        return '🟡 Moderate Risk'

merged_df['removal_risk'] = merged_df.apply(risk_score, axis=1)

### Step 34: Priority Scoring Engine

In [ ]:
def priority_score(row):
    score = 0
    if row['action'].startswith('🔁'):
        score += 40
    if row['confidence'] == '🔵 High':
        score += 30
    if row['removal_risk'] == '✅ Low Risk to Remove':
        score += 20
    if row['max_recovery'] > 3000:
        score += 10
    return score

merged_df['priority_score'] = merged_df.apply(priority_score, axis=1)

### Step 35: Final Executive SKU Action Playbook

In [ ]:
executive_playbook = merged_df[[
    'itemid', 'action', 'max_recovery', 'confidence',
    'removal_risk', 'priority_score'
]].sort_values(by='priority_score', ascending=False)

executive_playbook.head(10)

### Step 36: Business Impact Simulation – Delay vs Inaction


In [ ]:
# Assume promo delay = 15% loss in recovery, no action = 50% erosion growth
merged_df['recovery_if_now'] = merged_df['max_recovery']
merged_df['recovery_if_delayed'] = merged_df['max_recovery'] * 0.85
merged_df['loss_if_ignored'] = merged_df['max_recovery'] * 1.5

### Step 37: Trade-off Penalty Estimation

In [ ]:
def tradeoff_penalty(row):
    if row['action'].startswith('🔁'):
        return round(row['purchases_x'] * 0.1, 1)  # assume 10% drop
    else:
        return 0

merged_df['expected_purchase_loss'] = merged_df.apply(tradeoff_penalty, axis=1)

### Step 38: Risk-Adjusted Recovery Score

In [ ]:
# Map labels to weights
confidence_map = {'🔵 High': 1.0, '🟡 Medium': 0.75, '🔴 Low': 0.5}
risk_map = {'✅ Low Risk to Remove': 0.1, '🟡 Moderate Risk': 0.3, '⚠️ High Risk to Remove': 0.6}

merged_df['confidence_score'] = merged_df['confidence'].map(confidence_map)
merged_df['risk_score'] = merged_df['removal_risk'].map(risk_map)

# Risk-adjusted recovery
merged_df['adjusted_recovery'] = merged_df['max_recovery'] * merged_df['confidence_score'] * (1 - merged_df['risk_score'])

### Step 39: Final Strategic Action Table

In [ ]:
final_strategy_table = merged_df[[
    'itemid', 'action', 'max_recovery', 'confidence', 'removal_risk',
    'expected_purchase_loss', 'adjusted_recovery', 'priority_score'
]].sort_values(by='adjusted_recovery', ascending=False)

final_strategy_table.head(10)

### Step 40: Save & Confirm Final Output

In [ ]:
# Save merged_df as CSV into the output folder
output_path = "/kaggle/working/merged_strategy_dataset.csv"
merged_df.to_csv(output_path, index=False)

# Confirm it's saved by listing files
import os
os.listdir("/kaggle/working")

### Step 41: Save Final Strategy Dataset as CSV

In [ ]:
# Save your final merged DataFrame to a downloadable CSV
merged_df.to_csv("/kaggle/working/merged_strategy_dataset.csv", index=False)

# Check if the file exists
import os
print(os.listdir("/kaggle/working"))

### Step 42: Classify Product Lifecycle Stages

In [ ]:
# Step 1: Define lifecycle classification function
def classify_lifecycle(row):
    if row['views_x'] < 300 and row['purchases_x'] == 0:
        return '🍼 Early Stage'
    elif row['purchases_x'] > 5 and row['erosion_flag'] == 0:
        return '🚀 Growing'
    elif row['erosion_flag'] == 1 and row['priority_score'] < 50:
        return '🧊 Declining'
    elif row['purchases_x'] == 0 and row['gross_margin'] == 0:
        return '💀 Dead-weight'
    else:
        return '📦 Stable/Unclassified'

# Step 2: Apply to your dataset
merged_df['lifecycle_stage'] = merged_df.apply(classify_lifecycle, axis=1)

# Step 3: Group by lifecycle stage
lifecycle_summary = merged_df.groupby('lifecycle_stage').agg(
    sku_count=('itemid', 'count'),
    total_margin=('gross_margin', 'sum'),
    avg_priority_score=('priority_score', 'mean'),
    avg_views=('views_x', 'mean'),
    avg_margin_per_unit=('margin_per_unit_x', 'mean')
).reset_index()

# Step 4: Display
lifecycle_summary

### Step 43: Final Executive Strategy Sheet

In [ ]:
# Select and rename relevant columns
exec_summary = merged_df[[
    'itemid', 'action', 'adjusted_recovery', 'confidence', 'removal_risk',
    'expected_purchase_loss', 'priority_score', 'lifecycle_stage'
]].copy()

# Rename columns for exec readability
exec_summary.columns = [
    'SKU ID', 'Recommended Action', 'Adjusted Recovery ($)', 'Confidence Level',
    'Removal Risk', 'Expected Purchase Loss', 'Priority Score', 'Lifecycle Stage'
]

# Sort by Priority Score (high to low)
exec_summary = exec_summary.sort_values(by='Priority Score', ascending=False).reset_index(drop=True)

# Display summary
exec_summary.head(20)  # or display the full DataFrame

### Step 44: Business Impact Simulator

In [ ]:
# Simulate outcomes for different actions
simulator = exec_summary.copy()

# Assume business logic for simulation:
# Keep = no recovery
# Cut Promo = adjusted recovery added to savings
# Remove = adjusted recovery + expected_purchase_loss avoided

simulator['Simulated Margin Recovered ($)'] = simulator.apply(
    lambda row: row['Adjusted Recovery ($)'] if 'Cut Promo' in row['Recommended Action'] else 0, axis=1
)

# Aggregate total recovery
total_recovery = simulator['Simulated Margin Recovered ($)'].sum()

# Optional: Breakdown by lifecycle
impact_by_stage = simulator.groupby('Lifecycle Stage')['Simulated Margin Recovered ($)'].sum().reset_index()

# Show simulator results
print("💰 Total Margin Recovered if Actions Are Implemented: ${:,.2f}".format(total_recovery))
impact_by_stage

In [ ]:
simulator = exec_summary.copy()
simulator['Simulated Margin Recovered ($)'] = simulator.apply(
    lambda row: row['Adjusted Recovery ($)'] if 'Cut Promo' in row['Recommended Action'] else 0, axis=1
)
total_recovery = simulator['Simulated Margin Recovered ($)'].sum()
impact_by_stage = simulator.groupby('Lifecycle Stage')['Simulated Margin Recovered ($)'].sum().reset_index()

## Step 45: Integrate Simulated Recovery into Executive Sheet

In [ ]:
# Add the simulation column to the executive summary
exec_summary['Simulated Margin Recovered ($)'] = simulator['Simulated Margin Recovered ($)']

## Step 46: Export Final Executive Strategy Sheet

In [ ]:
exec_summary.to_csv("final_strategy_master_sheet.csv", index=False)

## Step 47: Curate Top 5 High-Impact Erosion SKUs for Executive Summary

In [ ]:
# Assign the data to a DataFrame named 'final_strategy_summary'
final_strategy_summary = pd.DataFrame({
    'itemid': [5411, 187946, 111530, 370653, 91755],
    'views': [2325.0, 3410.0, 1397.0, 1854.0, 1024.0],
    'purchases': [0.0, 0.0, 11.0, 0.0, 0.0],
    'margin_per_unit': [38.57, 20.33, 38.30, 19.60, 30.06],
    'view_to_buy': [0.0, 0.0, 0.007874, 0.0, 0.0],
    'potential_extra_purchases': [65.640964, 96.273414, 28.441044, 52.343376, 28.910257],
    'potential_margin_recovered': [2531.771998, 1957.238517, 1089.291985, 1025.930160, 869.042326]
})

In [ ]:
# This will show all your current DataFrame variables
for var_name in dir():
    try:
        if isinstance(eval(var_name), pd.DataFrame):
            print(var_name)
    except:
        pass

##  Step 49: Extract & Pivot Key Item Metadata for Enrichment

In [ ]:
import pandas as pd

# Define properties you care about
important_props = {'categoryid', 'brand', 'color', 'name'}

def load_important_properties(filepath, important_props, chunk_size=50000):
    chunks = []
    for chunk in pd.read_csv(filepath, chunksize=chunk_size):
        chunk = chunk[chunk['property'].isin(important_props)]
        chunks.append(chunk)
    return pd.concat(chunks, ignore_index=True)

# Load both item property parts
props1_filtered = load_important_properties('/kaggle/input/ecommerce-dataset/item_properties_part1.csv', important_props)
props2_filtered = load_important_properties('/kaggle/input/ecommerce-dataset/item_properties_part2.csv', important_props)

# Combine and clean
filtered_props = pd.concat([props1_filtered, props2_filtered], ignore_index=True)
filtered_props['timestamp'] = pd.to_datetime(filtered_props['timestamp'])

# Keep latest property value per (itemid, property)
latest_filtered = filtered_props.sort_values('timestamp').drop_duplicates(['itemid', 'property'], keep='last')

# Pivot to wide format — each property becomes a column
metadata_df = latest_filtered.pivot(index='itemid', columns='property', values='value').reset_index()

# View result
metadata_df.head()

## Step 50: Clean and Enrich Category Tree

In [ ]:
# Load category tree safely
cat_tree = pd.read_csv('/kaggle/input/ecommerce-dataset/category_tree.csv')

# Check current columns
print(cat_tree.columns)

# Rename them correctly — based on your file it looks like:
# 1st col = itemid, 2nd col = categoryid
cat_tree.columns = ['itemid', 'categoryid']

# OPTIONAL: If you want to add a readable category name, you can later map this using a dictionary:
category_map = {
    '1038': 'Footwear',
    '1171': 'Mobile Phones',
    '1305': 'T-Shirts',
    '1114': 'Headphones',
    '209': 'Kitchenware',
    # Add more if needed
}

cat_tree['category_name'] = cat_tree['categoryid'].astype(str).map(category_map)

cat_tree.head()

## Step 51: Identify Available Metadata Properties

In [ ]:
# What properties are actually present?
filtered_props['property'].value_counts().head(20)

## Step 52: Add Human-Readable Category Labels to Metadata

In [ ]:
# Ensure categoryid is string
metadata_df['categoryid'] = metadata_df['categoryid'].astype(str)

# Map human-readable category labels
category_map = {
    '213': 'Laptops',
    '169': 'Electronics',
    '9': 'Books',
    '885': 'Men\'s Clothing',
    '1691': 'Mobiles',
    '1038': 'Footwear',
    '1171': 'Smartphones',
    '1305': 'T-Shirts',
    '1114': 'Headphones',
    '209': 'Kitchenware',
    # Add more as needed
}

metadata_df['category_name'] = metadata_df['categoryid'].map(category_map)

# Preview the cleaned metadata
metadata_df[['itemid', 'categoryid', 'category_name']].head()

## Step 53: Load Sample Rows and Inspect Column Structures from Final Datasets

In [ ]:
import pandas as pd

# Load only first few rows of each
merged_strategy = pd.read_csv('/kaggle/input/final-dataset/merged_strategy_dataset.csv', nrows=5)
profit_erosion = pd.read_csv('/kaggle/input/profit-erosion/profit_erosion_sku_analysis.csv', nrows=5)
uplift_df = pd.read_csv('/kaggle/input/uplift-data/uplift_final.csv', nrows=5)

# Show column names
print("Merged Strategy Columns:\n", merged_strategy.columns.tolist())
print("Profit Erosion Columns:\n", profit_erosion.columns.tolist())
print("Uplift Columns:\n", uplift_df.columns.tolist())

## Step 54: Enrich Strategy Dataset with Category Metadata

In [ ]:
import pandas as pd

# Load merged strategy dataset
merged_strategy = pd.read_csv('/kaggle/input/final-dataset/merged_strategy_dataset.csv')

# Ensure itemid is string in both
metadata_df['itemid'] = metadata_df['itemid'].astype(str)
merged_strategy['itemid'] = merged_strategy['itemid'].astype(str)

# Merge on itemid to add category info
enriched_df = merged_strategy.merge(
    metadata_df[['itemid', 'categoryid', 'category_name']], 
    on='itemid', how='left'
)

# Preview key columns
enriched_df[['itemid', 'views_x', 'purchases_x', 'margin_per_unit_x', 'category_name']].head()

## Step 55: Extract and Pivot Category Metadata for Each SKU



In [ ]:
# Step 1: Load important properties using smart chunking
import pandas as pd

important_props = {'categoryid'}

def load_properties(filepath, props_to_keep):
    chunks = []
    for chunk in pd.read_csv(filepath, chunksize=50000):
        chunk = chunk[chunk['property'].isin(props_to_keep)]
        chunks.append(chunk)
    return pd.concat(chunks, ignore_index=True)

props1 = load_properties('/kaggle/input/ecommerce-dataset/item_properties_part1.csv', important_props)
props2 = load_properties('/kaggle/input/ecommerce-dataset/item_properties_part2.csv', important_props)

all_props = pd.concat([props1, props2])
all_props['timestamp'] = pd.to_datetime(all_props['timestamp'])

# Step 2: Keep latest property value for each itemid
latest_props = all_props.sort_values('timestamp').drop_duplicates(['itemid', 'property'], keep='last')

# Step 3: Pivot categoryid into column
metadata_df = latest_props.pivot(index='itemid', columns='property', values='value').reset_index()
metadata_df['itemid'] = metadata_df['itemid'].astype(str)

## Step 56: Merge Category Metadata into Final Strategy Dataset

In [ ]:
# Load main strategy dataset
merged_strategy = pd.read_csv('/kaggle/input/final-dataset/merged_strategy_dataset.csv')
merged_strategy['itemid'] = merged_strategy['itemid'].astype(str)

# Merge category info
enriched_df = merged_strategy.merge(metadata_df[['itemid', 'categoryid']], on='itemid', how='left')

## Step 57: Map Category IDs to Readable Names

In [ ]:
# Map readable names
category_map = {
    '213': 'Laptops', '169': 'Electronics', '9': 'Books',
    '885': 'Men\'s Clothing', '1691': 'Mobiles',
    '1038': 'Footwear', '1171': 'Smartphones', '1305': 'T-Shirts',
    '1114': 'Headphones', '209': 'Kitchenware'
}

enriched_df['category_name'] = enriched_df['categoryid'].astype(str).map(category_map)

## Step 58: Check How Many Products Have Valid Category Names

In [ ]:
# How many products now have a valid category_name?
matched = enriched_df['category_name'].notna().sum()
total = len(enriched_df)

print(f"✅ Category matched for {matched} out of {total} SKUs")

# Show sample of matched rows
enriched_df[enriched_df['category_name'].notna()].head(10)

## Step 59: Extract Unique Item IDs from Strategy Dataset

In [ ]:
# Get unique itemids from strategy dataset
strategy_items = merged_strategy['itemid'].astype(str).unique().tolist()

## Step 60: Load Relevant Properties Filtered by Item IDs and Property

In [ ]:
important_props = {'categoryid'}

def load_relevant_properties(filepath, itemids, props_to_keep):
    chunks = []
    for chunk in pd.read_csv(filepath, chunksize=50000):
        chunk = chunk[(chunk['property'].isin(props_to_keep)) & (chunk['itemid'].isin(itemids))]
        chunks.append(chunk)
    return pd.concat(chunks, ignore_index=True)

# Load only relevant categoryid rows
props1 = load_relevant_properties('/kaggle/input/ecommerce-dataset/item_properties_part1.csv', strategy_items, important_props)
props2 = load_relevant_properties('/kaggle/input/ecommerce-dataset/item_properties_part2.csv', strategy_items, important_props)

filtered_props = pd.concat([props1, props2])
filtered_props['timestamp'] = pd.to_datetime(filtered_props['timestamp'])

# Keep latest per item
latest = filtered_props.sort_values('timestamp').drop_duplicates(['itemid', 'property'], keep='last')

# Pivot to get categoryid column
metadata_df = latest.pivot(index='itemid', columns='property', values='value').reset_index()
metadata_df['itemid'] = metadata_df['itemid'].astype(str)

## Step 61: Reload Filtered Properties with Correct Item ID Type and Show Sample


In [ ]:
# Step 1: Define the filtering function again
def load_relevant_properties(filepath, itemids, props_to_keep):
    chunks = []
    for chunk in pd.read_csv(filepath, chunksize=50000):
        chunk = chunk[(chunk['property'].isin(props_to_keep)) & (chunk['itemid'].isin(itemids))]
        chunks.append(chunk)
    return pd.concat(chunks, ignore_index=True)

# Step 2: Extract itemids from your strategy dataset
strategy_items = merged_strategy['itemid'].astype(int).unique().tolist()  # Use int if original file uses int

# Step 3: Now call the function properly
important_props = {'categoryid'}
props1 = load_relevant_properties('/kaggle/input/ecommerce-dataset/item_properties_part1.csv', strategy_items, important_props)
props2 = load_relevant_properties('/kaggle/input/ecommerce-dataset/item_properties_part2.csv', strategy_items, important_props)

# Step 4: Show what we got
print("Filtered rows from props1:", len(props1))
print("Filtered rows from props2:", len(props2))
print("Sample props1 rows:\n", props1.head())

## Step 62: Combine Filtered Properties and Extract Latest Category per Item

In [ ]:
import pandas as pd

# Combine and convert timestamp
all_filtered = pd.concat([props1, props2])
all_filtered['timestamp'] = pd.to_datetime(all_filtered['timestamp'], unit='ms')

# Sort and drop duplicates to get latest categoryid per item
latest_props = all_filtered.sort_values('timestamp').drop_duplicates(['itemid'], keep='last')

# Create metadata_df with itemid and categoryid
metadata_df = latest_props[['itemid', 'value']].rename(columns={'value': 'categoryid'})
metadata_df['itemid'] = metadata_df['itemid'].astype(str)

## Step 63: Merge Category Data into Strategy Dataset

In [ ]:
merged_strategy['itemid'] = merged_strategy['itemid'].astype(str)

# Merge to get categoryid
enriched_df = merged_strategy.merge(metadata_df, on='itemid', how='left')

## Step 64: Map Category IDs to Readable Names



In [ ]:
# Map readable names
category_map = {
    '213': 'Laptops', '169': 'Electronics', '9': 'Books',
    '885': 'Men\'s Clothing', '1691': 'Mobiles',
    '1038': 'Footwear', '1171': 'Smartphones', '1305': 'T-Shirts',
    '1114': 'Headphones', '209': 'Kitchenware', '238': 'Tablets',
    '720': 'Fitness Gear', '819': 'Home Décor', '1613': 'Backpacks'
}

enriched_df['category_name'] = enriched_df['categoryid'].astype(str).map(category_map)

# Check how many got matched
print(f"✅ Category matched for {enriched_df['category_name'].notna().sum()} out of {len(enriched_df)} SKUs")

# Preview matched rows
enriched_df[enriched_df['category_name'].notna()].head()

## Step 65: Aggregate and Summarize by Category

In [ ]:
# Group by category to see cumulative impact
category_summary = enriched_df[enriched_df['category_name'].notna()].groupby('category_name').agg(
    sku_count=('itemid', 'count'),
    total_views=('views_x', 'sum'),
    total_purchases=('purchases_x', 'sum'),
    avg_margin=('margin_per_unit_x', 'mean'),
    total_recovery=('adjusted_recovery', 'sum'),
    avg_priority=('priority_score', 'mean')
).reset_index().sort_values(by='total_recovery', ascending=False)

category_summary

# Step 66: Prepare Executive SKU Strategy View

In [ ]:
# Select and rename key columns for executive view
sku_strategy = enriched_df[[
    'itemid', 'views_x', 'purchases_x', 'margin_per_unit_x', 'view_to_buy_x',
    'adjusted_recovery', 'priority_score', 'action', 'confidence', 'removal_risk'
]].copy()

# Rename for readability
sku_strategy.columns = [
    'SKU ID', 'Views', 'Purchases', 'Margin/Unit', 'View-to-Buy Rate',
    'Adjusted Recovery ($)', 'Priority Score', 'Recommended Action',
    'Confidence Level', 'Removal Risk'
]

# Sort by Priority Score + Recovery
sku_strategy = sku_strategy.sort_values(by=['Priority Score', 'Adjusted Recovery ($)'], ascending=[False, False])

# Display Top 10
sku_strategy.head(10)

# Step 67: Format and finalize SKU strategy table for presentation

In [ ]:
# Round key values for presentation
sku_strategy['View-to-Buy Rate'] = sku_strategy['View-to-Buy Rate'].round(4)
sku_strategy['Adjusted Recovery ($)'] = sku_strategy['Adjusted Recovery ($)'].round(2)
sku_strategy['Margin/Unit'] = sku_strategy['Margin/Unit'].round(2)

# Reorder columns for maximum clarity
sku_strategy = sku_strategy[[
    'SKU ID', 'Views', 'Purchases', 'Margin/Unit', 'View-to-Buy Rate',
    'Adjusted Recovery ($)', 'Recommended Action', 'Confidence Level',
    'Removal Risk', 'Priority Score'
]]

# Sort by priority and margin
sku_strategy = sku_strategy.sort_values(by=['Priority Score', 'Adjusted Recovery ($)'], ascending=[False, False])

# Preview final table
sku_strategy.head(10)

# Step 68: Simulate realistic pricing and cost data by category

In [ ]:
# Example: Category-wise price rules (realistic estimates)
price_rules = {
    'Headphones': (30, 150),
    'Home Décor': (20, 100),
    'Smartphones': (150, 1000),
    'T-Shirts': (10, 40),
    'Footwear': (25, 120),
    'Backpacks': (20, 80),
    'Kitchenware': (10, 60),
    'Books': (5, 25)
}

import numpy as np

# Assign realistic prices based on category
def simulate_price(row):
    cat = row['category_name']
    if cat in price_rules:
        return round(np.random.uniform(*price_rules[cat]), 2)
    else:
        return round(np.random.uniform(20, 80), 2)  # fallback

# Apply pricing logic
enriched_df['simulated_price'] = enriched_df.apply(simulate_price, axis=1)
enriched_df['simulated_cost'] = (enriched_df['simulated_price'] * np.random.uniform(0.6, 0.8)).round(2)
enriched_df['margin_per_unit'] = enriched_df['simulated_price'] - enriched_df['simulated_cost']

# Step 69: Train XGBoost classifier on profit erosion dataset and plot top feature importance


In [ ]:
from xgboost import XGBClassifier, plot_importance
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Prepare data
X = profit_erosion.drop(columns=['erosion_flag', 'itemid'])
y = profit_erosion['erosion_flag']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

# Fit model
model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
model.fit(X_train, y_train)

# Plot feature importance
plt.figure(figsize=(10,6))
plot_importance(model, importance_type='gain', max_num_features=10)
plt.title("Top Features Influencing Erosion Risk")
plt.show()

# Step 70: Train Logistic Regression and plot feature importance with seaborn

In [ ]:
from sklearn.linear_model import LogisticRegression
import seaborn as sns
import matplotlib.pyplot as plt

model = LogisticRegression()
model.fit(X, y)

coef_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': model.coef_[0]
}).sort_values(by='Importance', key=abs, ascending=False)

sns.barplot(data=coef_df, x='Importance', y='Feature')
plt.title('Feature Importance from Logistic Regression')
plt.show()

# Step 71: Train XGBoost on full dataset and plot top feature importance


In [ ]:
import pandas as pd
from xgboost import XGBClassifier, plot_importance
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Load full dataset
profit_erosion_full = pd.read_csv('/kaggle/input/profit-erosion/profit_erosion_sku_analysis.csv')

# Drop rows with missing target
profit_erosion_full = profit_erosion_full.dropna(subset=['erosion_flag'])

# Prepare features and labels
X = profit_erosion_full.drop(columns=['erosion_flag', 'itemid'])
y = profit_erosion_full['erosion_flag']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

# Train model
model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
model.fit(X_train, y_train)

# Plot feature importance
plt.figure(figsize=(10, 6))
plot_importance(model, importance_type='gain', max_num_features=10)
plt.title("Top Features Influencing Erosion Risk")
plt.show()

# Step 72: Extract and export feature importance scores for Tableau visualization


In [ ]:
import pandas as pd
from xgboost import plot_importance

# Get feature scores from the trained model
importance = model.get_booster().get_score(importance_type='gain')

# Convert to DataFrame
feat_imp_df = pd.DataFrame.from_dict(importance, orient='index', columns=['importance']).reset_index()
feat_imp_df.columns = ['feature', 'importance']

# Sort by importance
feat_imp_df = feat_imp_df.sort_values(by='importance', ascending=False)

# Export to CSV for Tableau
feat_imp_df.to_csv('/kaggle/working/feature_importance_for_tableau.csv', index=False)

feat_imp_df.head(10)

In [ ]:
import pandas as pd

# Load all key datasets
strategy_df = pd.read_csv('/kaggle/input/final-dataset/merged_strategy_dataset.csv')
uplift_df = pd.read_csv('/kaggle/input/uplift-data/uplift_final.csv')
erosion_df = pd.read_csv('/kaggle/input/profit-erosion/profit_erosion_sku_analysis.csv')

# Merge strategy + uplift
merged_df = strategy_df.merge(uplift_df, on='itemid', how='left', suffixes=('', '_uplift'))

# Merge with erosion metrics
merged_df = merged_df.merge(erosion_df, on='itemid', how='left', suffixes=('', '_erosion'))

# Drop duplicate columns
merged_df = merged_df.loc[:, ~merged_df.columns.duplicated()]

# Export the final enriched dataset
merged_df.to_csv('/kaggle/working/final_enriched_strategy_dataset.csv', index=False)

print("✅ Final dataset saved with shape:", merged_df.shape)
merged_df.head()

In [ ]:
c=pd.read_csv("/kaggle/input/ecommerce-dataset/item_properties_part1.csv")
c.head()

# Step 73: Merge all key datasets into one enriched master dataset

In [ ]:
import pandas as pd
from scipy.stats import chi2_contingency, mannwhitneyu

# ✅ Step 1: Load the dataset
events = pd.read_csv('/kaggle/input/ecommerce-dataset/events.csv')

# ✅ Step 2: Tag users who made a purchase as Group A (Exposed), rest as Group B (Control)
purchase_users = set(events[events['event'] == 'transaction']['visitorid'])
events['group'] = events['visitorid'].apply(lambda x: 'A' if x in purchase_users else 'B')

# ✅ Step 3: Aggregate metrics per visitor
user_metrics = events.groupby(['visitorid', 'group'])['event'].value_counts().unstack(fill_value=0).reset_index()

# Add conversion flag
user_metrics['converted'] = user_metrics['transaction'] > 0

# ✅ Step 4: Run Chi-Square Test for Conversion Rate
conversion_table = pd.crosstab(user_metrics['group'], user_metrics['converted'])
chi2, pval, _, _ = chi2_contingency(conversion_table)

print("🔍 A/B Test - Conversion Rate (Chi-Square)")
print(conversion_table)
print(f"Chi2 Stat: {chi2:.4f}, p-value: {pval:.4f}")
print(f"📌 Statistically Significant? {'Yes' if pval < 0.05 else 'No'}")

# ✅ Step 5: Mann-Whitney U Test for Views Per User
views_A = user_metrics[user_metrics['group'] == 'A']['view']
views_B = user_metrics[user_metrics['group'] == 'B']['view']
stat, p = mannwhitneyu(views_A, views_B, alternative='two-sided')

print("\n📊 A/B Test - Views Per User (Mann-Whitney U)")
print(f"Group A (Exposed) Avg Views: {views_A.mean():.2f}")
print(f"Group B (Control) Avg Views: {views_B.mean():.2f}")
print(f"U Stat: {stat:.4f}, p-value: {p:.4f}")
print(f"📌 Statistically Significant? {'Yes' if p < 0.05 else 'No'}")

# Step 74: Perform A/B test analysis on user conversion and engagement


In [ ]:
import pandas as pd
from scipy.stats import chi2_contingency, mannwhitneyu, ttest_ind

# Load your dataset
df = pd.read_csv('/kaggle/input/final-dataset/merged_strategy_dataset.csv')

# Filter Groups
group_A = df[df['action'] == '🔁 Cut Promo (C)']
group_B = df[df['action'].isin(['⭕ No Action', None, ''])]

# Sanity check
print("Group A (Promo-Cut):", group_A.shape[0], "SKUs")
print("Group B (Control):", group_B.shape[0], "SKUs")

from scipy.stats import fisher_exact

# Prepare contingency table
contingency = [[converted_A, not_converted_A],
               [converted_B, not_converted_B]]

# Use Fisher's Exact Test instead of Chi-Square
odds_ratio, p_val_fisher = fisher_exact(contingency)

print("\n🔍 A/B Test - Conversion Rate (Fisher's Exact)")
print(f"Converted A: {converted_A}, Converted B: {converted_B}")
print(f"Odds Ratio: {odds_ratio:.4f}, p-value: {p_val_fisher:.4f}")
print("📌 Statistically Significant?", "Yes" if p_val_fisher < 0.05 else "No")

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt

# Load the dataset
df = pd.read_csv("/kaggle/input/final-dataset/merged_strategy_dataset.csv")

# Step 1: Define Features and Target
features = [
    'views_x', 'add_to_cart', 'purchases_x', 
    'view_to_cart', 'cart_to_buy', 'view_to_buy_x',
    'price', 'cost', 'margin_per_unit_x', 'gross_margin',
    'confidence_score', 'priority_score'
]

target = 'adjusted_recovery'

# Step 2: Drop rows with missing target
df_model = df.dropna(subset=[target])

# Step 3: Train-Test Split
X = df_model[features]
y = df_model[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 4: Train Model
model = GradientBoostingRegressor(random_state=42)
model.fit(X_train, y_train)

# Step 5: Predict
y_pred = model.predict(X_test)

# Step 6: Evaluation
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print(f"✅ RMSE: {rmse:.2f}")
print(f"✅ R² Score: {r2:.2f}")

# Step 7: Feature Importance
feature_importance = pd.DataFrame({
    'feature': features,
    'importance': model.feature_importances_
}).sort_values(by='importance', ascending=False)

print("\n📊 Top Predictive Features:")
print(feature_importance)

# Optional: Save predictions to original dataframe
df['predicted_recovery'] = model.predict(df[features])

# Optional: Export updated dataset
df.to_csv("final_with_predictions.csv", index=False)
print("✅ File saved as final_with_predictions.csv")

# Step 75: Generate smart strategic action recommendations based on priority, recovery, and confidence


In [ ]:
# Step 1: Normalize priority and predicted recovery
df['norm_priority'] = (df['priority_score'] - df['priority_score'].min()) / (df['priority_score'].max() - df['priority_score'].min())
df['norm_predicted_recovery'] = (df['predicted_recovery'] - df['predicted_recovery'].min()) / (df['predicted_recovery'].max() - df['predicted_recovery'].min())

# Step 2: Create Action Value Score (weighted)
df['action_value_score'] = (0.5 * df['norm_predicted_recovery']) + (0.3 * df['norm_priority']) + (0.2 * df['confidence_score'])

# Step 3: Define Confidence Filter
df['filtered_confidence'] = df['confidence_score'].apply(lambda x: 1 if x >= 0.5 else 0)

# Step 4: Smart Action Logic
def decide_action(row):
    if row['filtered_confidence'] == 0:
        return 'Ignore'
    elif row['removal_risk'] == 1:
        return 'Delay'
    elif row['action_value_score'] > 0.7:
        return 'Act Now'
    elif row['action_value_score'] > 0.4:
        return 'Cut'
    else:
        return 'Ignore'

df['smart_action'] = df.apply(decide_action, axis=1)

# Step 5: Simulated Gain if Acted
df['expected_gain_if_act'] = df.apply(lambda row: row['predicted_recovery'] if row['smart_action'] == 'Act Now' else 0, axis=1)

# Step 6: Save Updated File
df.to_csv("final_strategic_actions.csv", index=False)
print("✅ Strategic action file saved: final_strategic_actions.csv")

# Optional: See counts
print("\n📦 Action Breakdown:")
print(df['smart_action'].value_counts())